In [2]:
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\manas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
import json
import nltk
import numpy as np
import random  # Import the random module
from tensorflow.keras.models import load_model

# Load the intents JSON file
with open('D:\\DOCUMENT HDD\\P_project\\Intelligent customer support chatbot\\Chatbot using tensorflow\\intents.json') as file:
    intents = json.load(file)

lemmatizer = nltk.WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.']

# Tokenizing the words and creating classes
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatizing and lowering each word, and removing duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(set(words))

classes = sorted(set(classes))

# Load the model
model = load_model('chatbot_model.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("Chatbot is running!")

while True:
    message = input("You: ")
    ints = predict_class(message, model)
    res = get_response(ints, intents)
    print(f"Chatbot: {res}")


Chatbot is running!
1/1 [==============================] - 0s 67ms/step
Chatbot: Hi there, how can I help?
1/1 [==============================] - 0s 33ms/step
Chatbot: Have a nice day
1/1 [==============================] - 0s 33ms/step
Chatbot: My pleasure
1/1 [==============================] - 0s 26ms/step
Chatbot: Good to see you again
1/1 [==============================] - 0s 25ms/step
Chatbot: Hi there, how can I help?
1/1 [==============================] - 0s 28ms/step
Chatbot: Hello
1/1 [==============================] - 0s 20ms/step
Chatbot: Hello
1/1 [==============================] - 0s 34ms/step
Chatbot: Hello
1/1 [==============================] - 0s 31ms/step
Chatbot: Good to see you again


KeyboardInterrupt: Interrupted by user